In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [105]:
card_path = "card_stats_royalapi.csv"
deck_path = "Decks - Sheet1.csv"

card_stats = pd.read_csv(card_path)
decks = pd.read_csv(deck_path)

In [106]:
print("Brackets covered:", card_stats["Bracket"].unique())
n_unique_cards = card_stats["CardName"].nunique()
print("Unique cards:", n_unique_cards)
brackets_per_card = card_stats_agg.groupby("CardName")["Bracket"].nunique()
cards_all_four = brackets_per_card[brackets_per_card == 4].index.tolist()
print("Cards present in all four brackets:", len(cards_all_four))
card_stats_agg = (
card_stats
.groupby(["CardName", "Bracket"], as_index=False)
.agg({
"UsagePct": "mean",
"WinPct": "mean",
"Rating": "mean"
})
)

n_card_bracket = card_stats_agg.shape[0]
print("Unique card–bracket combos (after aggregation):", n_card_bracket)

Brackets covered: ['Top 200' 'Top 1000' 'All Ranked' 'Ladder']
Unique cards: 125
Cards present in all four brackets: 123
Unique card–bracket combos (after aggregation): 497


In [107]:
desc = (
card_stats_agg.groupby("Bracket")["UsagePct"]
.agg(["mean", "median", "std", "min", "max"])
.rename(columns={
"mean": "Mean",
"median": "Median",
"std": "Std",
"min": "Min",
"max": "Max"
})
)
print("\nUsagePct stats by bracket:")
print(np.round(desc,2))


UsagePct stats by bracket:
            Mean  Median    Std  Min   Max
Bracket                                   
All Ranked  5.75     4.0   8.95  0.0  88.0
Ladder      5.49     3.0   9.66  0.0  92.0
Top 1000    6.03     4.0   9.77  0.0  92.0
Top 200     6.10     3.5  10.08  0.0  93.0


In [108]:
def top_cards_by_usage(df, bracket, top_n=1):
  sub = df[df["Bracket"] == bracket].copy()
  sub = sub.sort_values("UsagePct", ascending=False)
  return sub[["CardName", "UsagePct", "WinPct"]].head(top_n)

print("\nTop 10 by UsagePct in Top 200:")
print(top_cards_by_usage(card_stats_agg, "Top 200", top_n=10))
print("\nTop 10 by UsagePct in Top 1000:")
print(top_cards_by_usage(card_stats_agg, "Top 1000", top_n=10))
print("\nTop 10 by UsagePct in Top Ladder:")
print(top_cards_by_usage(card_stats_agg, "Ladder", top_n=10))

print("\nTop 10 by UsagePct in All Ranked:")
print(top_cards_by_usage(card_stats_agg, "All Ranked", top_n=10))


Top 10 by UsagePct in Top 200:
             CardName  UsagePct  WinPct
460    Tower Princess      93.0    48.0
27   Barbarian Barrel      45.0    49.0
138          Fireball      31.0    48.0
110    Electro Spirit      26.0    48.0
468             Vines      21.0    49.0
420         Skeletons      19.0    49.0
314           Minions      17.0    49.0
274         Lightning      15.0    48.0
444           The Log      15.0    47.0
386       Royal Ghost      14.0    49.0

Top 10 by UsagePct in Top 1000:
             CardName  UsagePct  WinPct
459    Tower Princess      92.0    49.0
26   Barbarian Barrel      41.0    50.0
137          Fireball      31.0    49.0
109    Electro Spirit      25.0    50.0
419         Skeletons      18.5    49.5
443           The Log      18.0    49.0
313           Minions      17.0    49.0
467             Vines      17.0    49.0
10             Arrows      15.0    48.0
305    Mini P.E.K.K.A      14.0    49.0

Top 10 by UsagePct in Top Ladder:
           CardName 

In [109]:
corrs = {}

for bracket, grp in card_stats_agg.groupby("Bracket"):
    if grp["UsagePct"].nunique() > 1 and grp["WinPct"].nunique() > 1:
        corrs[bracket] = grp["UsagePct"].corr(grp["WinPct"])
    else:
        corrs[bracket] = np.nan

corrs = pd.Series(corrs, name="UsageWinCorr")

print("\nPearson correlation between UsagePct and WinPct (per bracket):")
print(corrs.round(2))


Pearson correlation between UsagePct and WinPct (per bracket):
All Ranked    0.04
Ladder        0.29
Top 1000      0.14
Top 200       0.14
Name: UsageWinCorr, dtype: float64


In [110]:
rows = []
for bracket, grp in card_stats_agg.groupby("Bracket"):
    grp = grp.sort_values("UsagePct", ascending=False)
    total_usage = grp["UsagePct"].sum()
    row = {"Bracket": bracket}
    for n in (5, 10, 20):
        top_usage = grp["UsagePct"].head(n).sum()
        if total_usage != 0:
            row[f"Top {n} share"] = 100 * top_usage / total_usage
        else:
            row[f"Top {n} share"] = np.nan
    rows.append(row)

meta_conc = pd.DataFrame(rows).set_index("Bracket")

print("\nShare of total usage by the most-used cards (in %):")
print(meta_conc.round(2))


Share of total usage by the most-used cards (in %):
            Top 5 share  Top 10 share  Top 20 share
Bracket                                            
All Ranked        27.00         36.88         49.69
Ladder            29.88         41.25         55.90
Top 1000          27.76         38.60         52.98
Top 200           28.80         39.47         54.00


In [111]:
usage_pivot = card_stats_agg.pivot_table(
    index="CardName",
    columns="Bracket",
    values="UsagePct",
    aggfunc="mean"
)
win_pivot = card_stats_agg.pivot_table(
    index="CardName",
    columns="Bracket",
    values="WinPct",
    aggfunc="mean"
)
if {"Top 200", "Ladder"}.issubset(usage_pivot.columns):
    usage_diff = (usage_pivot["Top 200"] - usage_pivot["Ladder"]).dropna()
    usage_diff = usage_diff.rename("Top200_minus_Ladder")

    largest_positive = usage_diff.sort_values(ascending=False).head(20)
    largest_negative = usage_diff.sort_values(ascending=True).head(20)

    print("\nLargest positive usage gaps (Top 200 vs Ladder):")
    print(largest_positive.round(2))

    print("\nLargest negative usage gaps (Top 200 vs Ladder):")
    print(largest_negative.round(2))
else:
    print("\nvalue missing")
if {"Top 200", "All Ranked"}.issubset(usage_pivot.columns):
    usage_diff = (usage_pivot["Top 200"] - usage_pivot["All Ranked"]).dropna()
    usage_diff = usage_diff.rename("Top200_minus_AllRanked")

    largest_positive = usage_diff.sort_values(ascending=False).head(20)
    largest_negative = usage_diff.sort_values(ascending=True).head(20)

    print("\nLargest positive usage gaps (Top 200 vs All Ranked):")
    print(largest_positive.round(2))

    print("\nLargest negative usage gaps (Top 200 vs All Ranked):")
    print(largest_negative.round(2))
else:
    print("\nvalue missing")

if {"Top 200", "Ladder"}.issubset(win_pivot.columns):
    win_diff = (win_pivot["Top 200"] - win_pivot["Ladder"]).dropna()
    win_diff = win_diff.rename("Top200_minus_Ladder")

    top200_better = win_diff.sort_values(ascending=False).head(20)
    top200_worse = win_diff.sort_values(ascending=True).head(20)

    print("\nCards where Top 200 win more than Ladder (Top 200 vs Ladder):")
    print(top200_better.round(2))

    print("\nCards where Top 200 win less than Ladder (Top 200 vs Ladder):")
    print(top200_worse.round(2))
else:
    print("\nvalue missing")
if {"Top 200", "All Ranked"}.issubset(win_pivot.columns):
    win_diff = (win_pivot["Top 200"] - win_pivot["All Ranked"]).dropna()
    win_diff = win_diff.rename("Top200_minus_AllRanked")

    top200_better = win_diff.sort_values(ascending=False).head(20)
    top200_worse = win_diff.sort_values(ascending=True).head(20)

    print("\nCards where Top 200 win more than All Ranked (Top 200 vs All Ranked):")
    print(top200_better.round(2))

    print("\nCards where Top 200 win less than All Ranked (Top 200 vs All Ranked):")
    print(top200_worse.round(2))
else:
    print("\nvalue missing")



Largest positive usage gaps (Top 200 vs Ladder):
CardName
Barbarian Barrel    37.0
Electro Spirit      21.0
Vines               15.0
Lightning           12.0
Fireball            11.0
Minions             10.0
Royal Ghost         10.0
Skeletons           10.0
Tornado              9.0
Zappies              9.0
Goblin Hut           7.0
Lava Hound           7.0
Golem                7.0
Fisherman            7.0
Ice Wizard           7.0
Executioner          6.5
Miner                5.0
Mega Minion          5.0
Baby Dragon          4.5
Ice Golem            4.0
Name: Top200_minus_Ladder, dtype: float64

Largest negative usage gaps (Top 200 vs Ladder):
CardName
The Log            -23.0
Witch              -15.0
Arrows             -14.0
Hog Rider          -14.0
Mega Knight        -13.5
Goblin Gang        -12.0
Mini P.E.K.K.A     -11.0
Firecracker        -10.0
Goblin Barrel       -9.5
Tesla               -7.5
Princess            -7.0
Wizard              -6.5
Knight              -6.0
Electro Wizard 

In [112]:

meta_cols = ["DeckName", "Bracket", "WinPct", "UsagePCT", "Rating"]
card_cols = [c for c in decks.columns if c not in meta_cols]

print("\nNumber of decks:", decks.shape[0])
deck_stats = decks[["WinPct", "UsagePCT", "Rating"]].agg(["mean", "min", "max"])
print("\nDeck-level stats (mean / min / max):")
print(deck_stats.round(2))


most_used_decks = (
    decks.sort_values("UsagePCT", ascending=False)[["DeckName", "WinPct", "UsagePCT"]]
    .head(5)
)
print("\nMost used decks (by UsagePCT):")
print(most_used_decks.round(2))


deck_cards_long = decks.melt(
    id_vars=meta_cols,
    value_vars=card_cols,
    var_name="Slot",
    value_name="CardName"
)


name_map = {
    "Log": "The Log",
    "Gaint Snowball": "Giant Snowball",
}
deck_cards_long["CardName"] = deck_cards_long["CardName"].replace(name_map)

print("\nNumber of card slots:", deck_cards_long.shape[0])
print("Unique cards across all decks:", deck_cards_long["CardName"].nunique())


card_counts = (
    deck_cards_long["CardName"]
    .value_counts()
    .rename_axis("CardName")
    .reset_index(name="Count")
)

print("\nTop 10 most common cards in the 29 decks:")
print(card_counts.head(10))


top10_cards = card_counts.head(10)["CardName"]

top200_stats = card_stats_agg[card_stats_agg["Bracket"] == "Top 200"][
    ["CardName", "UsagePct", "WinPct"]
].rename(
    columns={
        "UsagePct": "Top200_UsagePct",
        "WinPct": "Top200_WinPct",
    }
)

merged_freq_top200 = (
    card_counts[card_counts["CardName"].isin(top10_cards)]
    .merge(top200_stats, on="CardName", how="left")
)

print("\nTop 10 most common deck cards merged with Top 200 card stats:")
print(merged_freq_top200.sort_values("Count", ascending=False).round(2))



Number of decks: 29

Deck-level stats (mean / min / max):
      WinPct  UsagePCT  Rating
mean   51.18      0.97   51.45
min    42.20      0.10   30.00
max    55.10      3.70   61.00

Most used decks (by UsagePCT):
        DeckName  WinPct  UsagePCT
4   EvoRGFishboy    50.9       3.7
1       EvoRHogs    51.7       2.2
0    GolemEvoBBD    52.3       2.0
9       HogMM2.6    49.0       1.8
14  EvoRGFishboy    51.7       1.8

Number of card slots: 319
Unique cards across all decks: 148

Top 10 most common cards in the 29 decks:
           CardName  Count
0         Skeletons     20
1           The Log     13
2          Fireball     12
3  Barbarian Barrel     11
4        Ice Spirit     10
5            Cannon     10
6         Hog Rider      9
7    Electro Spirit      9
8         Musketeer      7
9         Ice Golem      7

Top 10 most common deck cards merged with Top 200 card stats:
           CardName  Count  Top200_UsagePct  Top200_WinPct
0         Skeletons     20             19.0        